## Install important libraries needed

In [57]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [58]:
df=pd.read_csv('data\StudentsPerformance.csv')
df.head(3)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


In [59]:
X=df.drop(columns='math score',axis=1)
y=df['math score']

In [60]:
X

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [61]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns

numeric_transformer=StandardScaler()
cat_transformer=OneHotEncoder()

preprocessor= ColumnTransformer(
    [
        ('num', numeric_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

In [62]:
X=preprocessor.fit_transform(X)


In [63]:
X.shape

(1000, 19)

In [64]:
x_train,x_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [75]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2_sq = r2_score(true,predicted)
    return mae,rmse,r2_sq

In [84]:
models={
    'LinearRegression' : LinearRegression(),
    'DecisionTreeRegressor' : DecisionTreeRegressor(),
    'RandomForestRegressor' : RandomForestRegressor(),
    'AdaBoostRegressor' : AdaBoostRegressor(),
    'Ridge' : Ridge(),
    'Lasso' : Lasso(),
    'CatBoostRegressor' : CatBoostRegressor(),
    'KNeighborsRegressor' : KNeighborsRegressor(),
    'XGBRegressor' : XGBRegressor()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    model_train_mae, model_train_rmse, model_train_r2=evaluate_model(y_train,y_train_pred)

    model_test_mae, model_test_rmse, model_test_r2=evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("model performance for train set")
    print(f'- rmse : {model_train_rmse:.4f}')
    print(f'- mae : {model_train_mae:.4f}')
    print(f'- r2 : {model_train_r2:.4f}')

    print('-------------------------------------------')

    print("model performance for test set")
    print(f'- rmse : {model_test_rmse:.4f}')
    print(f'- mae : {model_test_mae:.4f}')
    print(f'- r2 : {model_test_r2:.4f}')
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')




LinearRegression
model performance for train set
- rmse : 5.2972
- mae : 4.2383
- r2 : 0.8743
-------------------------------------------
model performance for test set
- rmse : 5.4825
- mae : 4.3379
- r2 : 0.8778


DecisionTreeRegressor
model performance for train set
- rmse : 0.2887
- mae : 0.0200
- r2 : 0.9996
-------------------------------------------
model performance for test set
- rmse : 7.7571
- mae : 5.9640
- r2 : 0.7554


RandomForestRegressor
model performance for train set
- rmse : 2.2678
- mae : 1.8139
- r2 : 0.9770
-------------------------------------------
model performance for test set
- rmse : 6.0273
- mae : 4.6878
- r2 : 0.8523


AdaBoostRegressor
model performance for train set
- rmse : 5.8500
- mae : 4.8053
- r2 : 0.8467
-------------------------------------------
model performance for test set
- rmse : 6.1571
- mae : 4.8228
- r2 : 0.8459


Ridge
model performance for train set
- rmse : 5.2976
- mae : 4.2368
- r2 : 0.8743
------------------------------------------

In [94]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['model_name','r2_score'])

,model_name,r2_score
0,LinearRegression,0.877824
1,DecisionTreeRegressor,0.755422
2,RandomForestRegressor,0.852336
3,AdaBoostRegressor,0.845909
4,Ridge,0.877990
5,Lasso,0.820028
6,CatBoostRegressor,0.854830
7,KNeighborsRegressor,0.794216
8,XGBRegressor,0.837904
